In [1]:
import cobra
from cobra import Metabolite, Gene, Reaction
from cobra.flux_analysis import single_gene_deletion
import pandas as pd
from medusa.test import load_universal_modelseed

In [2]:
#import the gapfilled version of the model for testing
model = cobra.io.read_sbml_model("../results/reconstructions/pst7.xml")
universal = load_universal_modelseed()

In [5]:
model.summary()

IN FLUXES            OUT FLUXES         OBJECTIVES
-------------------  -----------------  ------------
cpd00067_e  333      cpd10516_e  236    bio1  16.1
cpd10515_e  236      cpd00013_e   95.8
cpd00092_e   54.9    cpd00007_e   51.8
cpd00027_e   10      cpd00011_e   32.3
cpd00268_e    0.188  cpd00012_e    3.6
cpd00028_e    0.1
cpd00030_e    0.05
cpd00034_e    0.05
cpd00048_e    0.05
cpd00058_e    0.05
cpd00063_e    0.05
cpd00099_e    0.05
cpd00149_e    0.05
cpd00205_e    0.05
cpd00254_e    0.05


In [3]:
#for all exchange reactions, use the boundry reaction function in cobrapy, seems more straightforward
#define the media conditions, first the complete media conditions
# to do this set the exchange reactions open for constituents of the complete King's B media
lb = pd.read_csv("../data/lb_media.csv")
lb_ids= lb['ID'].tolist()
lb_medium = []
for met in lb_ids:
    met = met + "_e"
    lb_medium.append(met)
lb_medium

add_mets = []
add_exchanges = []
for met in lb_medium:
    try:
        model.metabolites.get_by_id(met)
    except:
        print('no '+met)
        add_met = universal.metabolites.get_by_id(met).copy()
        add_mets.append(add_met)
for met in lb_medium:
    # Search for exchange reactions
    try:
        model.reactions.get_by_id('EX_'+met)
    except:
        add_met = universal.metabolites.get_by_id(met)
        ex_rxn = Reaction('EX_' + met)
        ex_rxn.name = "Exchange reaction for " + met
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({add_met:-1})
        add_exchanges.append(ex_rxn)
        if add_met.id not in [m.id for m in add_mets]:
            add_mets.append(add_met)

no cpd00018_e
no cpd00033_e
no cpd00039_e
no cpd00046_e
no cpd00065_e
no cpd00066_e
no cpd00069_e
no cpd00084_e
no cpd00091_e
no cpd00126_e
no cpd00161_e
no cpd00182_e
no cpd00184_e
no cpd00215_e
no cpd00218_e
no cpd00220_e
no cpd00226_e
no cpd00239_e
no cpd00311_e
no cpd00383_e
no cpd00393_e
no cpd00438_e
no cpd00541_e
no cpd00644_e
no cpd00654_e
no cpd00793_e
no cpd01048_e
no cpd03424_e
no cpd11595_e


In [4]:
lb_medium_dict = {}
for met in lb_medium:
    lb_medium_dict[met] = 1000
print (lb_medium_dict)

{'cpd00001_e': 1000, 'cpd00007_e': 1000, 'cpd00009_e': 1000, 'cpd00018_e': 1000, 'cpd00023_e': 1000, 'cpd00027_e': 1000, 'cpd00028_e': 1000, 'cpd00030_e': 1000, 'cpd00033_e': 1000, 'cpd00034_e': 1000, 'cpd00035_e': 1000, 'cpd00039_e': 1000, 'cpd00041_e': 1000, 'cpd00046_e': 1000, 'cpd00048_e': 1000, 'cpd00051_e': 1000, 'cpd00054_e': 1000, 'cpd00058_e': 1000, 'cpd00060_e': 1000, 'cpd00063_e': 1000, 'cpd00065_e': 1000, 'cpd00066_e': 1000, 'cpd00067_e': 1000, 'cpd00069_e': 1000, 'cpd00084_e': 1000, 'cpd00091_e': 1000, 'cpd00092_e': 1000, 'cpd00099_e': 1000, 'cpd00107_e': 1000, 'cpd00119_e': 1000, 'cpd00126_e': 1000, 'cpd00129_e': 1000, 'cpd00149_e': 1000, 'cpd00156_e': 1000, 'cpd00161_e': 1000, 'cpd00182_e': 1000, 'cpd00184_e': 1000, 'cpd00205_e': 1000, 'cpd00215_e': 1000, 'cpd00218_e': 1000, 'cpd00220_e': 1000, 'cpd00226_e': 1000, 'cpd00239_e': 1000, 'cpd00246_e': 1000, 'cpd00249_e': 1000, 'cpd00254_e': 1000, 'cpd00311_e': 1000, 'cpd00322_e': 1000, 'cpd00383_e': 1000, 'cpd00393_e': 1000,

In [6]:
# Find metabolites from the biolog data that are missing in the test model
# and add them from the universal
missing_mets = []
missing_exchanges = []
media_dicts = {}

# remove duplicate conditions by using a set
for met_id in lb_medium:
    media_dict = lb_medium_dict.copy()
    
    for single_met_id in met_id.split(','):
        # make a boolean indicator for metabolites missing from both
        # the model and the universal model
        missing = False
        
        # add the _e suffix for extracellular metabolites
       # single_met_id = single_met_id ==
        
        # search for the metabolite in the model.
        # If missing, try to find it in the universal.
        if single_met_id in [m.id for m in model.metabolites]:
            met = model.metabolites.get_by_id(single_met_id)
        else:
            if single_met_id in [m.id for m in universal.metabolites]:
                met = universal.metabolites.get_by_id(single_met_id)
            else:
                print(single_met_id + ' not in universal. Ignoring metabolite.')
                missing = True
        
        # If the metabolite was in the universal or the model,
        # check for an existing exchange reaction. If not there,
        # create and add the exchange reaction.
        if not missing:
            if 'EX_' + single_met_id not in [rxn.id for rxn in model.reactions]:
                ex_rxn = Reaction('EX_' + single_met_id)
                ex_rxn.name = "Exchange reaction for " + single_met_id
                ex_rxn.lower_bound = -1000
                ex_rxn.upper_bound = 1000
                ex_rxn.add_metabolites({met:-1})
                if ex_rxn.id not in [r.id for r in missing_exchanges]:
                    missing_exchanges.append(ex_rxn)
            if met_id in media_dicts.keys():
                # if media dict was already there, it means this is a double C/N
                # case (E.g. D+L mets). Don't need to alter the rest of the dict.
                media_dict['EX_'+single_met_id] = 1000
            else:
                media_dict = {'EX_'+k:v for k,v in media_dict.items()}
                media_dict['EX_'+single_met_id] = 1000
            
            media_dicts[met_id] = media_dict
            print(met_id + ' was not missing')


cpd00001_e was not missing
cpd00007_e was not missing
cpd00009_e was not missing
cpd00018_e was not missing
cpd00023_e was not missing
cpd00027_e was not missing
cpd00028_e was not missing
cpd00030_e was not missing
cpd00033_e was not missing
cpd00034_e was not missing
cpd00035_e was not missing
cpd00039_e was not missing
cpd00041_e was not missing
cpd00046_e was not missing
cpd00048_e was not missing
cpd00051_e was not missing
cpd00054_e was not missing
cpd00058_e was not missing
cpd00060_e was not missing
cpd00063_e was not missing
cpd00065_e was not missing
cpd00066_e was not missing
cpd00067_e was not missing
cpd00069_e was not missing
cpd00084_e was not missing
cpd00091_e was not missing
cpd00092_e was not missing
cpd00099_e was not missing
cpd00107_e was not missing
cpd00119_e was not missing
cpd00126_e was not missing
cpd00129_e was not missing
cpd00149_e was not missing
cpd00156_e was not missing
cpd00161_e was not missing
cpd00182_e was not missing
cpd00184_e was not missing
c

In [7]:
# Add the exchange reactions for metabolites from biolog base
model.add_metabolites(add_mets)
print ("I have added" + str(add_mets))
model.add_reactions(add_exchanges)

# Do the same for single C/N supplements
model.add_metabolites(missing_mets)
model.add_reactions(missing_exchanges)

Ignoring reaction 'EX_cpd00018_e' since it already exists.
Ignoring reaction 'EX_cpd00033_e' since it already exists.
Ignoring reaction 'EX_cpd00039_e' since it already exists.
Ignoring reaction 'EX_cpd00046_e' since it already exists.
Ignoring reaction 'EX_cpd00065_e' since it already exists.
Ignoring reaction 'EX_cpd00066_e' since it already exists.
Ignoring reaction 'EX_cpd00069_e' since it already exists.
Ignoring reaction 'EX_cpd00084_e' since it already exists.
Ignoring reaction 'EX_cpd00091_e' since it already exists.
Ignoring reaction 'EX_cpd00126_e' since it already exists.
Ignoring reaction 'EX_cpd00161_e' since it already exists.
Ignoring reaction 'EX_cpd00182_e' since it already exists.
Ignoring reaction 'EX_cpd00184_e' since it already exists.
Ignoring reaction 'EX_cpd00215_e' since it already exists.
Ignoring reaction 'EX_cpd00218_e' since it already exists.
Ignoring reaction 'EX_cpd00220_e' since it already exists.
Ignoring reaction 'EX_cpd00226_e' since it already exist

I have added[<Metabolite cpd00018_e at 0x7fd071d4cc18>, <Metabolite cpd00033_e at 0x7fd071d4cf28>, <Metabolite cpd00039_e at 0x7fd071d4cfd0>, <Metabolite cpd00046_e at 0x7fd0c57bc198>, <Metabolite cpd00065_e at 0x7fd0c57bc2e8>, <Metabolite cpd00066_e at 0x7fd0c57bc828>, <Metabolite cpd00069_e at 0x7fd0c57bcac8>, <Metabolite cpd00084_e at 0x7fd0c57bcc18>, <Metabolite cpd00091_e at 0x7fd0c57bcda0>, <Metabolite cpd00126_e at 0x7fd071d42358>, <Metabolite cpd00161_e at 0x7fd071d424a8>, <Metabolite cpd00182_e at 0x7fd071d42588>, <Metabolite cpd00184_e at 0x7fd071d426a0>, <Metabolite cpd00215_e at 0x7fd071d428d0>, <Metabolite cpd00218_e at 0x7fd071d42a58>, <Metabolite cpd00220_e at 0x7fd071d42ba8>, <Metabolite cpd00226_e at 0x7fd071d42cc0>, <Metabolite cpd00239_e at 0x7fd071d42e10>, <Metabolite cpd00311_e at 0x7fd0c57c8080>, <Metabolite cpd00383_e at 0x7fd0c57c8400>, <Metabolite cpd00393_e at 0x7fd0c57c8668>, <Metabolite cpd00438_e at 0x7fd0c57c8898>, <Metabolite cpd00541_e at 0x7fd0c57c8b38>

In [8]:
lb_media_exchanges = {}
for key,value in lb_medium_dict.items():
    key = "EX_" + key
    lb_media_exchanges[key] = value
#print (lb_media_exchanges)
lb_media_exchanges['EX_cpd00028_e'] = 1
model.medium = lb_media_exchanges

model.medium

{'EX_cpd00023_e': 1000,
 'EX_cpd00034_e': 1000,
 'EX_cpd00041_e': 1000,
 'EX_cpd00048_e': 1000,
 'EX_cpd00107_e': 1000,
 'EX_cpd00119_e': 1000,
 'EX_cpd00156_e': 1000,
 'EX_cpd00322_e': 1000,
 'EX_cpd00205_e': 1000,
 'EX_cpd00971_e': 1000,
 'EX_cpd00067_e': 1000,
 'EX_cpd00060_e': 1000,
 'EX_cpd00129_e': 1000,
 'EX_cpd00027_e': 1000,
 'EX_cpd00054_e': 1000,
 'EX_cpd00009_e': 1000,
 'EX_cpd00001_e': 1000,
 'EX_cpd01012_e': 1000,
 'EX_cpd00058_e': 1000,
 'EX_cpd10516_e': 1000,
 'EX_cpd00531_e': 1000,
 'EX_cpd00149_e': 1000,
 'EX_cpd00035_e': 1000,
 'EX_cpd00051_e': 1000,
 'EX_cpd00249_e': 1000,
 'EX_cpd00007_e': 1000,
 'EX_cpd00030_e': 1000,
 'EX_cpd00063_e': 1000,
 'EX_cpd00099_e': 1000,
 'EX_cpd00254_e': 1000,
 'EX_cpd10515_e': 1000,
 'EX_cpd00028_e': 1,
 'EX_cpd00246_e': 1000,
 'EX_cpd00092_e': 1000,
 'EX_cpd00018_e': 1000,
 'EX_cpd00033_e': 1000,
 'EX_cpd00039_e': 1000,
 'EX_cpd00046_e': 1000,
 'EX_cpd00065_e': 1000,
 'EX_cpd00066_e': 1000,
 'EX_cpd00069_e': 1000,
 'EX_cpd00084_e': 1

In [9]:
#under complete media conditions, run single gene deletions and check for growth
deletion_results_complete_media = single_gene_deletion(model)

#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    

In [10]:
print (deletion_results_complete_media)

                     growth   status
ids                                 
(PSPTO_2373)   3.725284e-11  optimal
(PSPTO_1744)   3.725284e-11  optimal
(PSPTO_3757)   3.655367e-11  optimal
(PSPTO_0750)   3.655367e-11  optimal
(PSPTO_1692)   3.655367e-11  optimal
(PSPTO_1131)   3.655367e-11  optimal
(SPONTANEOUS)  3.655367e-11  optimal
(PSPTO_4144)   3.655367e-11  optimal
(PSPTO_2119)   1.141087e-12  optimal
(PSPTO_5338)   1.141087e-12  optimal
(PSPTO_4908)   1.141087e-12  optimal
(PSPTO_5277)   7.926434e-13  optimal
(PSPTO_5302)   7.926434e-13  optimal
(PSPTO_0604)   2.673177e-12  optimal
(PSPTO_4709)   2.673177e-12  optimal
(PSPTO_1747)   2.673177e-12  optimal
(PSPTO_1207)   2.673177e-12  optimal
(PSPTO_4254)   2.673177e-12  optimal
(PSPTO_4459)   2.673177e-12  optimal
(PSPTO_4290)   2.673177e-12  optimal
(PSPTO_1294)   2.673177e-12  optimal
(PSPTO_0233)   2.673177e-12  optimal
(PSPTO_1434)   2.673177e-12  optimal
(PSPTO_0224)   2.673177e-12  optimal
(PSPTO_5154)   2.673177e-12  optimal
(

In [ ]:
#define the media conditions, now in the Ara media defined by Anderson in the apoplast sampling paper, first Mock apoplasmic fluid
#again, this will be done by opening up exchange reactions for the metabolites that are present in the analysis
for met in model.metabolites:
    try  

In [ ]:
#under mock apoplasmic fluid media conditions, run single gene deletions and check for growth
deletion_results_ara_mock_media = single_gene_deletion(model)
#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    

In [ ]:
#define the media conditions, now in the Ara media defined by Anderson in the apoplast sampling paper, now Flg22 apoplasmic fluid
#again, this will be done by opening up exchange reactions for the metabolites that are present in the analysis
for met in model.metabolites:
    try  

In [ ]:
#under flg apoplasmic fluid media conditions, run single gene deletions and check for growth
deletion_results_ara_flg_media = single_gene_deletion(model)
#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    